In [1]:
library(Boruta)
library(dplyr)
library(readr)
library(randomForest)
library(pROC)

Warning message:
"package 'Boruta' was built under R version 4.4.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.


Attaching package: 'randomForest'


The following object is masked from 'package:dplyr':

    combine


Warning message:
"package 'pROC' was built under R version 4.4.3"
Type 'citation("pROC")' for a citation.


Attaching package: 'pROC'


The following objects are masked from 'package:stats':

    cov, smooth, var




In [2]:
##Read CSV file
whole_data <- read.csv("Soil_MetaFeature_Data.csv")

#Subset DataFrame to include only a specific genus at a time, replace as needed.
subset_data <- whole_data[whole_data$`FUNGI_Taxonomy` == 'Purepurocillium', ]

#Prints subset_data to verify what we're grabbing.
subset_data
dim(subset_data)

sampleid           Filename                Batch Type   InjOrder plate
1   p322_blue_E10_282  p322_blue_E10_282.mzML  NA    sample 282      p322 
2   p322_blue_E11_26   p322_blue_E11_26.mzML   NA    sample 26       p322 
3   p322_blue_E12_212  p322_blue_E12_212.mzML  NA    sample 212      p322 
4   p322_blue_E9_258   p322_blue_E9_258.mzML   NA    sample 258      p322 
5   p322_blue_F10_311  p322_blue_F10_311.mzML  NA    sample 311      p322 
6   p322_blue_F11_332  p322_blue_F11_332.mzML  NA    sample 332      p322 
7   p322_blue_F12_62   p322_blue_F12_62.mzML   NA    sample 62       p322 
8   p322_blue_F1_229   p322_blue_F1_229.mzML   NA    sample 229      p322 
9   p322_blue_F2_60    p322_blue_F2_60.mzML    NA    sample 60       p322 
10  p322_blue_F3_340   p322_blue_F3_340.mzML   NA    sample 340      p322 
11  p322_blue_F4_81    p322_blue_F4_81.mzML    NA    sample 81       p322 
12  p322_blue_F5_41    p322_blue_F5_41.mzML    NA    sample 41       p322 
13  p322_blue_F6_100   p322_blue_F6_100.mzML   NA    sample 100      p322 
14  p322_blue_F7_330   p322_blue_F7_330.mzML   NA    sample 330      p322 
15  p322_blue_F8_331   p322_blue_F8_331.mzML   NA    sample 331      p322 
16  p322_blue_F9_46    p322_blue_F9_46.mzML    NA    sample 46       p322 
17  p322_blue_G10_273  p322_blue_G10_273.mzML  NA    sample 273      p322 
18  p322_blue_G11_104  p322_blue_G11_104.mzML  NA    sample 104      p322 
19  p322_blue_G12_173  p322_blue_G12_173.mzML  NA    sample 173      p322 
20  p322_blue_G1_112   p322_blue_G1_112.mzML   NA    sample 112      p322 
21  p322_blue_G2_151   p322_blue_G2_151.mzML   NA    sample 151      p322 
22  p322_blue_G3_56    p322_blue_G3_56.mzML    NA    sample 56       p322 
23  p322_blue_G4_162   p322_blue_G4_162.mzML   NA    sample 162      p322 
24  p322_blue_G5_54    p322_blue_G5_54.mzML    NA    sample 54       p322 
25  p322_blue_G6_236   p322_blue_G6_236.mzML   NA    sample 236      p322 
26  p322_blue_G7_238   p322_blue_G7_238.mzML   NA    sample 238      p322 
27  p322_blue_G8_30    p322_blue_G8_30.mzML    NA    sample 30       p322 
28  p322_blue_G9_183   p322_blue_G9_183.mzML   NA    sample 183      p322 
29  p323_green_E10_324 p323_green_E10_324.mzML NA    sample 324      p323 
30  p323_green_E11_289 p323_green_E11_289.mzML NA    sample 289      p323 
⋮   ⋮                  ⋮                       ⋮     ⋮      ⋮        ⋮    
304 p332_green_G8_1276 p332_green_G8_1276.mzML NA    sample 1276     p332 
305 p332_green_G9_1121 p332_green_G9_1121.mzML NA    sample 1121     p332 
306 p333_red_E10_1182  p333_red_E10_1182.mzML  NA    sample 1182     p333 
307 p333_red_E11_1327  p333_red_E11_1327.mzML  NA    sample 1327     p333 
308 p333_red_E12_1290  p333_red_E12_1290.mzML  NA    sample 1290     p333 
309 p333_red_E9_1366   p333_red_E9_1366.mzML   NA    sample 1366     p333 
310 p333_red_F10_1167  p333_red_F10_1167.mzML  NA    sample 1167     p333 
311 p333_red_F11_1124  p333_red_F11_1124.mzML  NA    sample 1124     p333 
312 p333_red_F12_1368  p333_red_F12_1368.mzML  NA    sample 1368     p333 
313 p333_red_F1_1221   p333_red_F1_1221.mzML   NA    sample 1221     p333 
314 p333_red_F2_1092   p333_red_F2_1092.mzML   NA    sample 1092     p333 
315 p333_red_F3_1130   p333_red_F3_1130.mzML   NA    sample 1130     p333 
316 p333_red_F4_1097   p333_red_F4_1097.mzML   NA    sample 1097     p333 
317 p333_red_F5_1210   p333_red_F5_1210.mzML   NA    sample 1210     p333 
318 p333_red_F6_1163   p333_red_F6_1163.mzML   NA    sample 1163     p333 
319 p333_red_F7_1251   p333_red_F7_1251.mzML   NA    sample 1251     p333 
320 p333_red_F8_1176   p333_red_F8_1176.mzML   NA    sample 1176     p333 
321 p333_red_F9_1320   p333_red_F9_1320.mzML   NA    sample 1320     p333 
322 p333_red_G10_1119  p333_red_G10_1119.mzML  NA    sample 1119     p333 
323 p333_red_G11_1240  p333_red_G11_1240.mzML  NA    sample 1240     p333 
324 p333_red_G12_1319  p333_red_G12_1319.mzML  NA    sample 1319     p333 
325 p333_red_G1_1226   p333_r

[1]   333 16375

In [3]:
##################
######332v323#####
##################

#Preamble prep stuff for later, change the columns being analyzed and groups being analyzed as needed.

#Further subsets to only include groups of interest.
subset_data_plate_1 <- subset_data %>%
  filter(plate %in% c('p332', 'p323'))

subset_data_plate_1

X_p323 <- subset_data_plate_1[, 12:16374]
y_p323 <- as.factor(subset(subset_data_plate_1, plate %in% c("p332", "p323"))$plate)

set.seed(123)          #Seed set for reproducibility

sampleid                         Filename                              Batch
1  p323_green_E10_324               p323_green_E10_324.mzML               NA   
2  p323_green_E11_289               p323_green_E11_289.mzML               NA   
3  p323_green_E12_167               p323_green_E12_167.mzML               NA   
4  p323_green_E9_312                p323_green_E9_312.mzML                NA   
5  p323_green_F10_280               p323_green_F10_280.mzML               NA   
6  p323_green_F11_299               p323_green_F11_299.mzML               NA   
7  p323_green_F12_116               p323_green_F12_116.mzML               NA   
8  p323_green_F1_111_20240117113329 p323_green_F1_111_20240117113329.mzML NA   
9  p323_green_F2_32                 p323_green_F2_32.mzML                 NA   
10 p323_green_F3_254                p323_green_F3_254.mzML                NA   
11 p323_green_F4_13                 p323_green_F4_13.mzML                 NA   
12 p323_green_F5_208                p323_green_F5_208.mzML                NA   
13 p323_green_F6_42                 p323_green_F6_42.mzML                 NA   
14 p323_green_F8_84                 p323_green_F8_84.mzML                 NA   
15 p323_green_F9_103                p323_green_F9_103.mzML                NA   
16 p323_green_G10_327               p323_green_G10_327.mzML               NA   
17 p323_green_G11_25                p323_green_G11_25.mzML                NA   
18 p323_green_G12_102               p323_green_G12_102.mzML               NA   
19 p323_green_G1_335_20240117143901 p323_green_G1_335_20240117143901.mzML NA   
20 p323_green_G2_146                p323_green_G2_146.mzML                NA   
21 p323_green_G3_215                p323_green_G3_215.mzML                NA   
22 p323_green_G4_109                p323_green_G4_109.mzML                NA   
23 p323_green_G5_168                p323_green_G5_168.mzML                NA   
24 p323_green_G6_66                 p323_green_G6_66.mzML                 NA   
25 p323_green_G7_217                p323_green_G7_217.mzML                NA   
26 p323_green_G8_252                p323_green_G8_252.mzML                NA   
27 p323_green_G9_96                 p323_green_G9_96.mzML                 NA   
28 p332_green_E10_1348              p332_green_E10_1348.mzML              NA   
29 p332_green_E11_1314              p332_green_E11_1314.mzML              NA   
30 p332_green_E12_1191              p332_green_E12_1191.mzML              NA   
31 p332_green_E9_1336               p332_green_E9_1336.mzML               NA   
32 p332_green_F10_1305              p332_green_F10_1305.mzML              NA   
33 p332_green_F11_1324              p332_green_F11_1324.mzML              NA   
34 p332_green_F12_1140              p332_green_F12_1140.mzML              NA   
35 p332_green_F1_1135               p332_green_F1_1135.mzML               NA   
36 p332_green_F2_1056               p332_green_F2_1056.mzML               NA   
37 p332_green_F3_1278               p332_green_F3_1278.mzML               NA   
38 p332_green_F4_1037               p332_green_F4_1037.mzML               NA   
39 p332_green_F5_1232               p332_green_F5_1232.mzML               NA   
40 p332_green_F6_1066               p332_green_F6_1066.mzML               NA   
41 p332_green_F7_1068               p332_green_F7_1068.mzML               NA   
42 p332_green_F8_1109               p332_green_F8_1109.mzML               NA   
43 p332_green_F9_1128               p332_green_F9_1128.mzML               NA   
44 p332_green_G10_1351              p332_green_G10_1351.mzML              NA   
45 p332_green_G11_1049              p332_green_G11_1049.mzML              NA   
46 p332_green_G12_1127              p332_green_G12_1127.mzML              NA   
47 p332_green_G1_1359               p332_green_G1_1359.mzML               NA   
48 p332_green_G2_1170               p332_green_G2_1170.mzML               NA   
49 p332_green_G3_1239               p332_green_G3_1239.mzML               NA   
50 

In [4]:
#Runs Boruta for p323vTarget plate group
boruta_p323 <- Boruta(X_p323, y_p323, doTrace = 2, maxRuns = 100) #100 is a hard cap. If it can't find anything by 100 runs, it'll stop.

##Get confirmed + tentative features
#Extract features (confirmed + tentative), this is needed for standard RF.
confirmed_features <- getSelectedAttributes(boruta_p323, withTentative = TRUE) 

#####Get importance scores (optional, we're not going off these values, but the final RF values).
boruta_stats <- attStats(boruta_p323)
boruta_filtered <- boruta_stats[confirmed_features, ]
boruta_filtered <- boruta_filtered[order(-boruta_filtered$meanImp), ]

##Write to CSV, again optional if you want to view this. Can change to print boruta_filtered.
#After this point, a new CSV was manually created from what was considered tentative or confirmed.
#I used our existing metadata and copied over the m/z featuredata for those features to make a new CSV.
#From that point on I continued using that new CSV (named 'Significant_Features_Preliminary_....).
write.csv(boruta_filtered, "New_7_LPS_Pur_18.csv", row.names = TRUE)

 1. run of importance source...

 2. run of importance source...

 3. run of importance source...

 4. run of importance source...

 5. run of importance source...

 6. run of importance source...

 7. run of importance source...

 8. run of importance source...

 9. run of importance source...

 10. run of importance source...

 11. run of importance source...

 12. run of importance source...

 13. run of importance source...

 14. run of importance source...

 15. run of importance source...

 16. run of importance source...

 17. run of importance source...

 18. run of importance source...

 19. run of importance source...

 20. run of importance source...

 21. run of importance source...

After 21 iterations, +47 secs: 

 confirmed 4 attributes: X321.1482_2.64_2406, X351.1342_3.99_12725, X593.3693_5.05_4757, X723.4317_5.91_8799;

 rejected 16192 attributes: X1001.4171_5.51_20947, X1002.1117_5.94_24872, X1002.1122_6.07_4961, X1003.3774_6.29_24744, X1003.6913_7.32_15561 and 16187 

In [5]:
##################
######329v326#####
##################

#Further subsets to only include groups of interest.
subset_data_plate_2 <- subset_data %>%
  filter(plate %in% c('p329', 'p326'))

subset_data_plate_2

X_p326 <- subset_data_plate_2[, 12:16374]
y_p326 <- as.factor(subset(subset_data_plate_2, plate %in% c("p329", "p326"))$plate)

set.seed(123)          #Seed set for reproducibility

sampleid            Filename                 Batch Type   InjOrder plate
1  p326_green_E11_631  p326_green_E11_631.mzML  NA    sample 631      p326 
2  p326_green_E12_509  p326_green_E12_509.mzML  NA    sample 509      p326 
3  p326_green_E9_653   p326_green_E9_653.mzML   NA    sample 653      p326 
4  p326_green_F10_622  p326_green_F10_622.mzML  NA    sample 622      p326 
5  p326_green_F11_641  p326_green_F11_641.mzML  NA    sample 641      p326 
6  p326_green_F12_457  p326_green_F12_457.mzML  NA    sample 457      p326 
7  p326_green_F1_452   p326_green_F1_452.mzML   NA    sample 452      p326 
8  p326_green_F2_373   p326_green_F2_373.mzML   NA    sample 373      p326 
9  p326_green_F3_595   p326_green_F3_595.mzML   NA    sample 595      p326 
10 p326_green_F4_354   p326_green_F4_354.mzML   NA    sample 354      p326 
11 p326_green_F5_549   p326_green_F5_549.mzML   NA    sample 549      p326 
12 p326_green_F6_383   p326_green_F6_383.mzML   NA    sample 383      p326 
13 p326_green_F7_385   p326_green_F7_385.mzML   NA    sample 385      p326 
14 p326_green_F8_426   p326_green_F8_426.mzML   NA    sample 426      p326 
15 p326_green_F9_445   p326_green_F9_445.mzML   NA    sample 445      p326 
16 p326_green_G10_668  p326_green_G10_668.mzML  NA    sample 668      p326 
17 p326_green_G11_366  p326_green_G11_366.mzML  NA    sample 366      p326 
18 p326_green_G12_444  p326_green_G12_444.mzML  NA    sample 444      p326 
19 p326_green_G1_676   p326_green_G1_676.mzML   NA    sample 676      p326 
20 p326_green_G2_487   p326_green_G2_487.mzML   NA    sample 487      p326 
21 p326_green_G3_556   p326_green_G3_556.mzML   NA    sample 556      p326 
22 p326_green_G4_450   p326_green_G4_450.mzML   NA    sample 450      p326 
23 p326_green_G5_510   p326_green_G5_510.mzML   NA    sample 510      p326 
24 p326_green_G6_408   p326_green_G6_408.mzML   NA    sample 408      p326 
25 p326_green_G7_558   p326_green_G7_558.mzML   NA    sample 558      p326 
26 p326_green_G8_593   p326_green_G8_593.mzML   NA    sample 593      p326 
27 p326_green_G9_438   p326_green_G9_438.mzML   NA    sample 438      p326 
28 p329_green_E10_1007 p329_green_E10_1007.mzML NA    sample 1007     p329 
29 p329_green_E11_972  p329_green_E11_972.mzML  NA    sample 972      p329 
30 p329_green_E12_850  p329_green_E12_850.mzML  NA    sample 850      p329 
31 p329_green_E9_994   p329_green_E9_994.mzML   NA    sample 994      p329 
32 p329_green_F10_963  p329_green_F10_963.mzML  NA    sample 963      p329 
33 p329_green_F11_982  p329_green_F11_982.mzML  NA    sample 982      p329 
34 p329_green_F12_798  p329_green_F12_798.mzML  NA    sample 798      p329 
35 p329_green_F1_793   p329_green_F1_793.mzML   NA    sample 793      p329 
36 p329_green_F2_715   p329_green_F2_715.mzML   NA    sample 715      p329 
37 p329_green_F3_937   p329_green_F3_937.mzML   NA    sample 937      p329 
38 p329_green_F4_695   p329_green_F4_695.mzML   NA    sample 695      p329 
39 p329_green_F5_890   p329_green_F5_890.mzML   NA    sample 890      p329 
40 p329_green_F6_725   p329_green_F6_725.mzML   NA    sample 725      p329 
41 p329_green_F7_727   p329_green_F7_727.mzML   NA    sample 727      p329 
42 p329_green_F8_767   p329_green_F8_767.mzML   NA    sample 767      p329 
43 p329_green_F9_786   p329_green_F9_786.mzML   NA    sample 786      p329 
44 p329_green_G10_1010 p329_green_G10_1010.mzML NA    sample 1010     p329 
45 p329_green_G11_708  p329_green_G11_708.mzML  NA    sample 708      p329 
46 p329_green_G12_785  p329_green_G12_785.mzML  NA    sample 785      p329 
47 p329_green_G1_1018  p329_green_G1_1018.mzML  NA    sample 1018     p329 
48 p329_green_G2_829   p329_green_G2_829.mzML   NA    sample 829      p329 
49 p329_green_G3_897   p329_green_G3_897.mzML   NA    sample 897      p329 
50 p329_green_G4_791   p329_green_G4_791.mzML   NA    sample 791      p329 
51 p329_green_G5_851   p329_green_G5_851.mzML   NA    sample 851      p329 
52 p329_green_G6_749   p329_green_G6_749.mzML   NA 

In [6]:
#Run Boruta for p3XXvTarget plate group
boruta_p326 <- Boruta(X_p326, y_p326, doTrace = 2, maxRuns = 100) #100 is a hard cap. If it can't find anything by 100 runs, it'll stop.

##Get confirmed + tentative features
#Extract features (confirmed + tentative), this is needed for standard RF.
confirmed_features <- getSelectedAttributes(boruta_p326, withTentative = TRUE) 

#####Get importance scores (optional, we're not going off these values, but the final RF values).
boruta_stats <- attStats(boruta_p326)
boruta_filtered <- boruta_stats[confirmed_features, ]
boruta_filtered <- boruta_filtered[order(-boruta_filtered$meanImp), ]

##Write to CSV, again optional if you want to view this. Can change to print boruta_filtered.
#After this point, a new CSV was manually created from what was considered tentative or confirmed.
#I used our existing metadata and copied over the m/z featuredata for those features to make a new CSV.
#From that point on I continued using that new CSV (named 'Significant_Features_Preliminary_....).
write.csv(boruta_filtered, "New_14_LPS_Pur_18.csv", row.names = TRUE)

 1. run of importance source...

 2. run of importance source...

 3. run of importance source...

 4. run of importance source...

 5. run of importance source...

 6. run of importance source...

 7. run of importance source...

 8. run of importance source...

 9. run of importance source...

 10. run of importance source...

 11. run of importance source...

 12. run of importance source...

 13. run of importance source...

 14. run of importance source...

 15. run of importance source...

 16. run of importance source...

 17. run of importance source...

 18. run of importance source...

 19. run of importance source...

 20. run of importance source...

 21. run of importance source...

After 21 iterations, +41 secs: 

 confirmed 1 attribute: X263.0531_0.25_11773;

 rejected 16251 attributes: X1001.4171_5.51_20947, X1002.1117_5.94_24872, X1002.1122_6.07_4961, X1003.3774_6.29_24744, X1003.6913_7.32_15561 and 16246 more;

 still have 111 attributes left.


 22. run of importance

In [7]:
##################
######332v329#####
##################

#Preamble prep stuff for later, change the columns being analyzed and groups being analyzed as needed.

#Further subsets to only include groups of interest.
subset_data_plate_1 <- subset_data %>%
  filter(plate %in% c('p332', 'p329'))

subset_data_plate_1

X_p329 <- subset_data_plate_1[, 12:16374]
y_p329 <- as.factor(subset(subset_data_plate_1, plate %in% c("p332", "p329"))$plate)

set.seed(123)          #Seed set for reproducibility

sampleid            Filename                 Batch Type   InjOrder plate
1  p329_green_E10_1007 p329_green_E10_1007.mzML NA    sample 1007     p329 
2  p329_green_E11_972  p329_green_E11_972.mzML  NA    sample 972      p329 
3  p329_green_E12_850  p329_green_E12_850.mzML  NA    sample 850      p329 
4  p329_green_E9_994   p329_green_E9_994.mzML   NA    sample 994      p329 
5  p329_green_F10_963  p329_green_F10_963.mzML  NA    sample 963      p329 
6  p329_green_F11_982  p329_green_F11_982.mzML  NA    sample 982      p329 
7  p329_green_F12_798  p329_green_F12_798.mzML  NA    sample 798      p329 
8  p329_green_F1_793   p329_green_F1_793.mzML   NA    sample 793      p329 
9  p329_green_F2_715   p329_green_F2_715.mzML   NA    sample 715      p329 
10 p329_green_F3_937   p329_green_F3_937.mzML   NA    sample 937      p329 
11 p329_green_F4_695   p329_green_F4_695.mzML   NA    sample 695      p329 
12 p329_green_F5_890   p329_green_F5_890.mzML   NA    sample 890      p329 
13 p329_green_F6_725   p329_green_F6_725.mzML   NA    sample 725      p329 
14 p329_green_F7_727   p329_green_F7_727.mzML   NA    sample 727      p329 
15 p329_green_F8_767   p329_green_F8_767.mzML   NA    sample 767      p329 
16 p329_green_F9_786   p329_green_F9_786.mzML   NA    sample 786      p329 
17 p329_green_G10_1010 p329_green_G10_1010.mzML NA    sample 1010     p329 
18 p329_green_G11_708  p329_green_G11_708.mzML  NA    sample 708      p329 
19 p329_green_G12_785  p329_green_G12_785.mzML  NA    sample 785      p329 
20 p329_green_G1_1018  p329_green_G1_1018.mzML  NA    sample 1018     p329 
21 p329_green_G2_829   p329_green_G2_829.mzML   NA    sample 829      p329 
22 p329_green_G3_897   p329_green_G3_897.mzML   NA    sample 897      p329 
23 p329_green_G4_791   p329_green_G4_791.mzML   NA    sample 791      p329 
24 p329_green_G5_851   p329_green_G5_851.mzML   NA    sample 851      p329 
25 p329_green_G6_749   p329_green_G6_749.mzML   NA    sample 749      p329 
26 p329_green_G7_899   p329_green_G7_899.mzML   NA    sample 899      p329 
27 p329_green_G8_935   p329_green_G8_935.mzML   NA    sample 935      p329 
28 p329_green_G9_779   p329_green_G9_779.mzML   NA    sample 779      p329 
29 p332_green_E10_1348 p332_green_E10_1348.mzML NA    sample 1348     p332 
30 p332_green_E11_1314 p332_green_E11_1314.mzML NA    sample 1314     p332 
31 p332_green_E12_1191 p332_green_E12_1191.mzML NA    sample 1191     p332 
32 p332_green_E9_1336  p332_green_E9_1336.mzML  NA    sample 1336     p332 
33 p332_green_F10_1305 p332_green_F10_1305.mzML NA    sample 1305     p332 
34 p332_green_F11_1324 p332_green_F11_1324.mzML NA    sample 1324     p332 
35 p332_green_F12_1140 p332_green_F12_1140.mzML NA    sample 1140     p332 
36 p332_green_F1_1135  p332_green_F1_1135.mzML  NA    sample 1135     p332 
37 p332_green_F2_1056  p332_green_F2_1056.mzML  NA    sample 1056     p332 
38 p332_green_F3_1278  p332_green_F3_1278.mzML  NA    sample 1278     p332 
39 p332_green_F4_1037  p332_green_F4_1037.mzML  NA    sample 1037     p332 
40 p332_green_F5_1232  p332_green_F5_1232.mzML  NA    sample 1232     p332 
41 p332_green_F6_1066  p332_green_F6_1066.mzML  NA    sample 1066     p332 
42 p332_green_F7_1068  p332_green_F7_1068.mzML  NA    sample 1068     p332 
43 p332_green_F8_1109  p332_green_F8_1109.mzML  NA    sample 1109     p332 
44 p332_green_F9_1128  p332_green_F9_1128.mzML  NA    sample 1128     p332 
45 p332_green_G10_1351 p332_green_G10_1351.mzML NA    sample 1351     p332 
46 p332_green_G11_1049 p332_green_G11_1049.mzML NA    sample 1049     p332 
47 p332_green_G12_1127 p332_green_G12_1127.mzML NA    sample 1127     p332 
48 p332_green_G1_1359  p332_green_G1_1359.mzML  NA    sample 1359     p332 
49 p332_green_G2_1170  p332_green_G2_1170.mzML  NA    sample 1170     p332 
50 p332_green_G3_1239  p332_green_G3_1239.mzML  NA    sample 1239     p332 
51 p332_green_G4_1133  p332_green_G4_1133.mzML  NA    sample 1133     p332 
52 p332_green_G5_1192  p332_green_G5_1192.mzML  NA 

In [8]:
#Run Boruta for p322vTarget plate group
boruta_p329 <- Boruta(X_p329, y_p329, doTrace = 2, maxRuns = 100) #100 is a hard cap. If it can't find anything by 100 runs, it'll stop.

##Get confirmed + tentative features
#Extract features (confirmed + tentative), this is needed for standard RF.
confirmed_features <- getSelectedAttributes(boruta_p329, withTentative = TRUE) 

#####Get importance scores (optional, we're not going off these values, but the final RF values).
boruta_stats <- attStats(boruta_p329)
boruta_filtered <- boruta_stats[confirmed_features, ]
boruta_filtered <- boruta_filtered[order(-boruta_filtered$meanImp), ]

##Write to CSV, again optional if you want to view this. Can change to print boruta_filtered.
#After this point, a new CSV was manually created from what was considered tentative or confirmed.
#I used our existing metadata and copied over the m/z featuredata for those features to make a new CSV.
#From that point on I continued using that new CSV (named 'Significant_Features_Preliminary_....).
write.csv(boruta_filtered, "New_14_No_LPS_Pur.csv", row.names = TRUE)

 1. run of importance source...

 2. run of importance source...

 3. run of importance source...

 4. run of importance source...

 5. run of importance source...

 6. run of importance source...

 7. run of importance source...

 8. run of importance source...

 9. run of importance source...

 10. run of importance source...

 11. run of importance source...

 12. run of importance source...

 13. run of importance source...

 14. run of importance source...

 15. run of importance source...

 16. run of importance source...

 17. run of importance source...

 18. run of importance source...

 19. run of importance source...

 20. run of importance source...

 21. run of importance source...

After 21 iterations, +40 secs: 

 confirmed 4 attributes: X206.0462_0.25_2276, X300.2899_7.44_4820, X336.1669_3.15_5920, X801.5092_5.88_13156;

 rejected 16086 attributes: X1001.4171_5.51_20947, X1002.1117_5.94_24872, X1002.1122_6.07_4961, X1003.3774_6.29_24744, X1003.6913_7.32_15561 and 16081 